# 목차
- 주식 투자 관련 데이터를 수집한다
- 현재 시점으로 모든 종목코드를 가져온다
  - 교만하지 말고 항상 네이버에 물어보라
  - 감 잡고, 이제 하나씩
- 저장된 투자정보 불러오기
- 어떤 주식에 투자하고 싶은가
- 만약 여기에 투자했더라면, 수익률은?

# 주식 투자 관련 데이터를 수집한다

In [1]:
import os

if os.path.exists('stockinfo.db'):
    os.remove('stockinfo.db')


In [2]:
import sqlite3
conn = sqlite3.connect('stockinfo.db')
cursor = conn.cursor()

In [3]:
cursor.execute('''
CREATE TABLE tb_stockinfo (
    code VARCHAR NOT NULL PRIMARY KEY,
    comp_name VARCHAR
)
''')

In [4]:
cursor.execute("""
CREATE TABLE financial_statement (
    code VARCHAR NOT NULL,
    year_month VARCHAR NOT NULL,
    sales INTEGER,
    revenue INTEGER,
    profit INTEGER,
    profit_ratio REAL,
    net_profit_ratio REAL,
    ROE REAL,
    debt_ratio REAL,
    quick_ratio REAL,
    reserve_ratio REAL,
    EPS INTEGER,
    PER REAL,
    BPS INTEGER,
    PBR REAL,
    DPS INTEGER,
    dividend_yeild_ratio REAL,
    dividned_payout_ratio REAL,
    PRIMARY KEY (code, year_month)
)
""")

# 현재 시점으로 모든 종목코드를 가져온다

In [5]:
import numpy as np
import pandas as pd

In [6]:
df_stockcode = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]

In [7]:
df_stockcode.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,JS전선,5560,절연선 및 케이블 제조업,"선박선,고무선,전력선,통신선 제조",2007-11-12,12월,이익희,http://www.jscable.co.kr,충청남도
1,거북선2호,101380,NaN,운송장비(선박) 임대,2008-04-25,12월,신주선,NaN,부산광역시
2,거북선6호,114140,NaN,NaN,2009-10-01,12월,김연신,NaN,제주특별자치도
3,교보메리츠,64900,NaN,"부동산 투자,운용",2002-01-30,12월,김 상 진,NaN,서울특별시
4,국제관광공사,28780,NaN,NaN,1966-03-18,12월,NaN,NaN,NaN


In [8]:
for i in df_stockcode.index:
    comp_name = df_stockcode['회사명'][i]
    code = df_stockcode['종목코드'][i]
    
    query_str = f'''
    INSERT INTO tb_stockinfo(code, comp_name)
    VALUES("{code:06d}", "{comp_name}")
    '''
    
    cursor.execute(query_str)

In [9]:
conn.commit()

In [10]:
cursor.execute('select * from tb_stockinfo where comp_name = "삼성전자"')
print(cursor.fetchall())

[('005930', '삼성전자')]


# 모든 주식코드에 대한 투자정보를 가져온다

## 교만하지 말고, 항상 네이버에 물어보라

In [11]:
naverfin_url = 'https://finance.naver.com/item/main.nhn?code={}'

df = pd.read_html(naverfin_url.format('005930'), encoding='euc-kr')

In [12]:
df[3]

주요재무정보    최근 연간 실적                                       최근 분기 실적  \
       주요재무정보     2017.12     2018.12     2019.12  2020.12(E)    2019.03   
       주요재무정보      IFRS연결      IFRS연결      IFRS연결      IFRS연결     IFRS연결   
0         매출액  2395754.00  2437714.00  2304009.00  2316336.00  523855.00   
1        영업이익   536450.00   588867.00   277685.00   326326.00   62333.00   
2       당기순이익   421867.00   443449.00   217389.00   253762.00   50436.00   
3       영업이익률       22.39       24.16       12.05       14.09      11.90   
4        순이익률       17.61       18.19        9.44       10.96       9.63   
5   ROE(지배주주)       21.01       19.63        8.69        9.54      16.21   
6        부채비율       40.68       36.97       34.12         NaN      36.27   
7        당좌비율      181.61      204.12      233.57         NaN     209.25   
8         유보율    24536.12    27531.92    28856.02         NaN   27832.98   
9      EPS(원)     5421.00     6024.00     3166.00     3696.00     752.00   
10     PER(배)        9.40        6.42       17.63       14.26       8.54   
11     BPS(원)    30427.00    35342.00    37528.00    39978.00   36142.00   
12     PBR(배)        1.67        1.09        1.49        1.32       1.24   
13   주당배당금(원)      850.00     1416.00     1416.00     1536.00        NaN   
14   시가배당률(%)        1.67        3.66        2.54         NaN        NaN   
15    배당성향(%)       14.09       21.92       44.73         NaN        NaN   

                                                           
      2019.06    2019.09    2019.12    2020.03 2020.06(E)  
       IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결  
0   561271.00  620035.00  598848.00  553252.00  511401.00  
1    65971.00   77779.00   71603.00   64473.00   64703.00  
2    51806.00   62877.00   52270.00   48849.00   52746.00  
3       11.75      12.54      11.96      11.65      12.65  
4        9.23      10.14       8.73       8.83      10.31  
5       13.23      10.05       8.69       8.45        NaN  
6       33.05      34.14      34.12      34.19        NaN  
7      230.74     235.80     233.57     237.80        NaN  
8    28129.35   28541.64   28856.02   29134.12        NaN  
9      746.00     899.00     770.00     720.00     735.00  
10      10.48      13.73      17.63      15.24      72.90  
11   36789.00   37600.00   37528.00   38053.00        NaN  
12       1.28       1.30       1.49       1.25        NaN  
13        NaN        NaN        NaN        NaN        NaN  
14        NaN        NaN        NaN        NaN        NaN  
15        NaN        NaN        NaN        NaN        NaN

In [13]:
for f in df[3].columns:
    print(f)

('주요재무정보', '주요재무정보', '주요재무정보')
('최근 연간 실적', '2017.12', 'IFRS연결')
('최근 연간 실적', '2018.12', 'IFRS연결')
('최근 연간 실적', '2019.12', 'IFRS연결')
('최근 연간 실적', '2020.12(E)', 'IFRS연결')
('최근 분기 실적', '2019.03', 'IFRS연결')
('최근 분기 실적', '2019.06', 'IFRS연결')
('최근 분기 실적', '2019.09', 'IFRS연결')
('최근 분기 실적', '2019.12', 'IFRS연결')
('최근 분기 실적', '2020.03', 'IFRS연결')
('최근 분기 실적', '2020.06(E)', 'IFRS연결')


In [14]:
df_finstat = df[3]
for col in df_finstat.columns:
    cur_col = df_finstat[col]
    print(cur_col[0:3])

0      매출액
1     영업이익
2    당기순이익
Name: (주요재무정보, 주요재무정보, 주요재무정보), dtype: object
0    2395754.0
1     536450.0
2     421867.0
Name: (최근 연간 실적, 2017.12, IFRS연결), dtype: float64
0    2437714.0
1     588867.0
2     443449.0
Name: (최근 연간 실적, 2018.12, IFRS연결), dtype: float64
0    2304009.0
1     277685.0
2     217389.0
Name: (최근 연간 실적, 2019.12, IFRS연결), dtype: float64
0    2316336.0
1     326326.0
2     253762.0
Name: (최근 연간 실적, 2020.12(E), IFRS연결), dtype: float64
0    523855.0
1     62333.0
2     50436.0
Name: (최근 분기 실적, 2019.03, IFRS연결), dtype: float64
0    561271.0
1     65971.0
2     51806.0
Name: (최근 분기 실적, 2019.06, IFRS연결), dtype: float64
0    620035.0
1     77779.0
2     62877.0
Name: (최근 분기 실적, 2019.09, IFRS연결), dtype: float64
0    598848.0
1     71603.0
2     52270.0
Name: (최근 분기 실적, 2019.12, IFRS연결), dtype: float64
0    553252.0
1     64473.0
2     48849.0
Name: (최근 분기 실적, 2020.03, IFRS연결), dtype: float64
0    511401.0
1     64703.0
2     52746.0
Name: (최근 분기 실적, 2020.06(E), IFRS연결)

## 감 잡고, 이제 하나씩

In [15]:
query = 'SELECT * FROM tb_stockinfo'
cursor.execute(query)
all_stockinfo = cursor.fetchall()

In [16]:
all_stockinfo[:5]

[('005560', 'JS전선'),
 ('101380', '거북선2호'),
 ('114140', '거북선6호'),
 ('064900', '교보메리츠'),
 ('028780', '국제관광공사')]

In [17]:
import math

def fill_table_with(code: str, df_finstat: object):
    
    for col in df_finstat.columns[1:4]:
        
        if type(col) == int:
            continue
        ym = col[1]
        if ':' in ym:
            continue
        
        cur_col = df_finstat[col]
        if len(cur_col) != 16:
            continue

        values = []
        for c in cur_col:
            try:
                val = float(c)
                if math.isnan(val):
                    val = 'NULL'
            except:
                val = 'NULL'
                
            values.append(val)

        sales, revenue, profit = values[:3]
        profit_ratio, net_profit_ratio, ROE = values[3:6]
        debt_ratio, quick_ratio, reserve_ratio = values[6:9]
        EPS, PER, BPS, PBR = values[9:13]
        DPS, dividend_yeild_ratio, dividned_payout_ratio = values[13:]

        query = f'''
        INSERT INTO financial_statement (
            code, year_month,
            sales, revenue, profit,
            profit_ratio, net_profit_ratio, ROE,
            debt_ratio, quick_ratio, reserve_ratio,
            EPS, PER, BPS, PBR,
            DPS, dividend_yeild_ratio, dividned_payout_ratio            
        ) VALUES (
            {code}, {ym},
            {sales}, {revenue}, {profit},
            {profit_ratio}, {net_profit_ratio}, {ROE},
            {debt_ratio}, {quick_ratio}, {reserve_ratio},
            {EPS}, {PER}, {BPS}, {PBR},
            {DPS}, {dividend_yeild_ratio}, {dividned_payout_ratio}
        )
        '''
        #print(query)
        cursor.execute(query)
        
    conn.commit()


In [18]:
import datetime as dt

starttime = dt.datetime.now()

for idx, info in enumerate(all_stockinfo):
    code, _ = info
    
    try:
        df_temp = pd.read_html(naverfin_url.format(code), encoding='euc-kr')
    except:
        continue
        
    if len(df_temp) < 3:
        continue
        
    df_finstat = df_temp[3]
    fill_table_with(code, df_finstat)
    
endtime = dt.datetime.now()

print(endtime - starttime)

0:13:48.733099


In [19]:
df = pd.read_sql_query("SELECT * FROM financial_statement", conn)
df.head()

,code,year_month,sales,revenue,profit,profit_ratio,net_profit_ratio,ROE,debt_ratio,quick_ratio,reserve_ratio,EPS,PER,BPS,PBR,DPS,dividend_yeild_ratio,dividned_payout_ratio
0,155660,2017.12,2264.0,139.0,110.0,6.14,4.87,8.15,62.04,66.07,1640.60,692,7.47,8721,0.59,50.0,0.97,7.22
1,155660,2018.12,2360.0,99.0,80.0,4.20,3.40,5.54,52.59,59.87,1727.00,494,8.07,9127,0.44,50.0,1.25,10.12
2,155660,2019.12,2363.0,107.0,88.0,4.51,3.72,5.83,54.77,75.34,1820.15,546,8.40,9613,0.48,50.0,1.09,9.15
3,78930,2017.12,155801.0,20408.0,10919.0,13.10,7.01,13.42,127.72,52.58,2190.74,10414,5.97,81650,0.76,1800.0,2.89,17.29
4,78930,2018.12,177444.0,22098.0,10305.0,12.45,5.81,11.14,115.02,51.75,2342.86,9537,5.41,89680,0.58,1900.0,3.68,19.93


In [20]:
conn.close()

# 저장된 투자정보를 불러오기

In [21]:
import sqlite3

conn = sqlite3.connect("stockinfo.db")
cursor = conn.cursor()

In [22]:
query = 'SELECT * FROM financial_statement'
df = pd.read_sql_query(query, conn)
df.head()

,code,year_month,sales,revenue,profit,profit_ratio,net_profit_ratio,ROE,debt_ratio,quick_ratio,reserve_ratio,EPS,PER,BPS,PBR,DPS,dividend_yeild_ratio,dividned_payout_ratio
0,155660,2017.12,2264.0,139.0,110.0,6.14,4.87,8.15,62.04,66.07,1640.60,692,7.47,8721,0.59,50.0,0.97,7.22
1,155660,2018.12,2360.0,99.0,80.0,4.20,3.40,5.54,52.59,59.87,1727.00,494,8.07,9127,0.44,50.0,1.25,10.12
2,155660,2019.12,2363.0,107.0,88.0,4.51,3.72,5.83,54.77,75.34,1820.15,546,8.40,9613,0.48,50.0,1.09,9.15
3,78930,2017.12,155801.0,20408.0,10919.0,13.10,7.01,13.42,127.72,52.58,2190.74,10414,5.97,81650,0.76,1800.0,2.89,17.29
4,78930,2018.12,177444.0,22098.0,10305.0,12.45,5.81,11.14,115.02,51.75,2342.86,9537,5.41,89680,0.58,1900.0,3.68,19.93


In [23]:
query = '''
SELECT
    INFO.comp_name AS `회사이름`
    , INFO.code AS `종목코드`
    , FS2019.EPS AS `EPS 2019`
    , FS2018.EPS AS `EPS 2018`
    , FS2017.EPS AS `EPS 2017`
    , FS2019.ROE AS `ROE 2019`
    , FS2018.ROE AS `ROE 2018`
    , FS2017.ROE AS `ROE 2017`
    , FS2019.net_profit_ratio AS `순이익률`
    
FROM
    tb_stockinfo INFO


JOIN
    financial_statement FS2019
ON
    INFO.code = FS2019.code
AND
    FS2019.year_month = "2019.12"
    

JOIN
    financial_statement FS2018
ON
    INFO.code = FS2018.code
AND
    FS2018.year_month = "2018.12"
    

JOIN
    financial_statement FS2017
ON
    INFO.code = FS2017.code
AND
    FS2017.year_month = "2017.12"

'''

In [24]:
df_screen = pd.read_sql_query(query, conn)
df_screen = df_screen.dropna()
df_screen

,회사이름,종목코드,EPS 2019,EPS 2018,EPS 2017,ROE 2019,ROE 2018,ROE 2017,순이익률
0,DSR,155660,546,494,692,5.83,5.54,8.15,3.72
2,락앤락,115390,302,554,634,2.44,4.68,5.33,3.37
3,삼성바이오로직스,207940,3067,3387,-1466,4.77,5.51,-2.41,28.92
4,선진,136490,899,754,2879,6.72,5.61,20.98,2.00
5,인터지스,129260,-902,-454,202,-14.20,-6.38,2.88,-9.21
7,컨버즈,109070,-5557,-12919,-18446,-548.06,-428.05,-129.14,-22.54
8,코오롱인더,120110,1092,2250,4830,1.58,3.04,6.49,0.50
9,한국철강,104700,414,264,-942,2.65,1.70,-5.83,2.65
10,한미약품,128940,4400,2102,5103,7.30,3.52,8.82,5.73
14,호전실업,111110,327,555,547,2.26,3.88,4.29,0.89


# 어떤 주식에 투자하고 싶은가

[종목 선정시 확인하여야 하는 최소한의 지표](https://to-get-her.tistory.com/464)

> Bottom-up 방식으로 수익성 지표를 고를 떄
> - EPS, 증가율 20% 이상
> - ROE, 최근 몇년 내 15% 이상
> - 순이익률/영업이익률, 15% 이상


In [25]:
df_screen['EPS INC 2019'] = df_screen['EPS 2019'] / df_screen['EPS 2018']
df_screen['EPS INC 2018'] = df_screen['EPS 2018'] / df_screen['EPS 2017']
df_screen.head()

,회사이름,종목코드,EPS 2019,EPS 2018,EPS 2017,ROE 2019,ROE 2018,ROE 2017,순이익률,EPS INC 2019,EPS INC 2018
0,DSR,155660,546,494,692,5.83,5.54,8.15,3.72,1.105263,0.713873
2,락앤락,115390,302,554,634,2.44,4.68,5.33,3.37,0.545126,0.873817
3,삼성바이오로직스,207940,3067,3387,-1466,4.77,5.51,-2.41,28.92,0.905521,-2.310368
4,선진,136490,899,754,2879,6.72,5.61,20.98,2.00,1.192308,0.261896
5,인터지스,129260,-902,-454,202,-14.20,-6.38,2.88,-9.21,1.986784,-2.247525


In [26]:
cond = (
    (df_screen['EPS INC 2019'] > 1.2) & (df_screen['EPS INC 2018'] > 1.2)
    & (df_screen['ROE 2019'] > 1.5) & (df_screen['ROE 2018'] > 1.5) & (df_screen["ROE 2017"] > 1.5)
    & (df_screen['순이익률'] > 1.5)
)

sum(cond)

23

In [28]:
df_screen = df_screen[cond]
df_screen

,회사이름,종목코드,EPS 2019,EPS 2018,EPS 2017,ROE 2019,ROE 2018,ROE 2017,순이익률,EPS INC 2019,EPS INC 2018
78,미원홀딩스,107590,6591,4154,3331,11.03,7.62,7.50,8.64,1.586663,1.247073
107,사람인에이치알,143240,2188,1433,293,25.35,20.17,4.56,27.21,1.526867,4.890785
112,슈프리마,236200,3643,1608,1284,21.18,11.06,9.80,35.97,2.265547,1.252336
149,테스나,131970,3079,2370,1313,17.50,21.87,14.33,22.12,1.299156,1.805027
164,샘표식품,248170,5870,3938,2517,18.57,14.58,10.51,9.55,1.490604,1.564561
186,램테크놀러지,171010,273,131,81,11.64,8.20,5.33,7.31,2.083969,1.617284
267,신흥에스이씨,243840,2446,1468,1002,17.29,12.52,9.95,6.69,1.666213,1.465070
281,에이치엘사이언스,239610,3839,2113,961,27.38,18.71,9.56,15.39,1.816848,2.198751
284,엔에스,217820,439,329,102,14.50,12.64,4.21,6.62,1.334347,3.225490
300,제노레이,122310,975,787,505,25.32,27.35,26.45,20.34,1.238882,1.558416


# 만약 여기에 투자했더라면, 수익률은?

In [126]:
naversise_url = "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"

pages = [1, 14]

In [127]:
price_gains = {}

for i in df_screen.index:
    code = df_screen['종목코드'][i]
    
    price_gains[code] = {}
    
    for p in pages:
        the_url = naversise_url.format(code, p)
        df = pd.read_html(the_url)[0]
        cond = (df['날짜'] == '2020.07.01') | (df['날짜'] == '2020.01.02')
        close_price = df[cond]['종가']

        try:
            price_gains[code][p] = close_price.values[0]
        except:
            price_gains[code][p] = None


In [131]:
df_screen['가격수익률%'] = 0

for i in df_screen.index:
    code = df_screen['종목코드'][i]
    
    if price_gains[code] and price_gains[code][14] is not None:
        df_screen['가격수익률%'][i] = price_gains[code][1] * 100 / price_gains[code][14]
        
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [132]:
df_screen

,회사이름,종목코드,EPS 2019,EPS 2018,EPS 2017,ROE 2019,ROE 2018,ROE 2017,순이익률,EPS INC 2019,EPS INC 2018,가격수익률%
78,미원홀딩스,107590,6591,4154,3331,11.03,7.62,7.50,8.64,1.586663,1.247073,108
107,사람인에이치알,143240,2188,1433,293,25.35,20.17,4.56,27.21,1.526867,4.890785,76
112,슈프리마,236200,3643,1608,1284,21.18,11.06,9.80,35.97,2.265547,1.252336,94
149,테스나,131970,3079,2370,1313,17.50,21.87,14.33,22.12,1.299156,1.805027,114
164,샘표식품,248170,5870,3938,2517,18.57,14.58,10.51,9.55,1.490604,1.564561,122
186,램테크놀러지,171010,273,131,81,11.64,8.20,5.33,7.31,2.083969,1.617284,98
267,신흥에스이씨,243840,2446,1468,1002,17.29,12.52,9.95,6.69,1.666213,1.465070,98
281,에이치엘사이언스,239610,3839,2113,961,27.38,18.71,9.56,15.39,1.816848,2.198751,106
284,엔에스,217820,439,329,102,14.50,12.64,4.21,6.62,1.334347,3.225490,130
300,제노레이,122310,975,787,505,25.32,27.35,26.45,20.34,1.238882,1.558416,44


# 실패한 내용들

In [12]:
fnguide_api = 'http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'

In [13]:
cursor.execute('select * from tb_stockinfo')
all_stockinfo = cursor.fetchall()

print(len(all_stockinfo))

3840


In [14]:
import math

price_cols = ['price', 'total', 'per', 'pbr']

def fill_tb_price(code: str, df_price: object):
    
    if len(df_price.index) != 4:
        #print("Not enough PRICE field ", code)
        return
        

    for i in df_price.columns:
        
        if i == ('IFRS 연결', 'IFRS 연결'):
            continue
            
        yyyymm = i[0]
        high = i[1]
        
        values = []
        for j in range(len(df_price[i])):
            val = 'NULL' if type(df_price[i][j]) != float or math.isnan(df_price[i][j]) else df_price[i][j]
            values.append(val)
            
        price, total, per, pbr = values
        #print(yyyymm, high, price, total, per, pbr)
        query = f'''
        INSERT INTO tb_price_indicator (
            code, year_month, high_low, price, total, per, pbr
        )
        VALUES (
            "{code}", "{yyyymm}", "{high}", {price}, {total}, {per}, {pbr}
        )
        '''
        cursor.execute(query)

    conn.commit()
    

In [15]:
invest_cols = [
    'code', 'year_month', 'eps', 'ebitdaps', 'cfps', 'sps', 'bps', 
    'dps', 'dps1', 'divpayout', 
    'per', 'pcr', 'psr', 'pbr', 'ev_sales', 'ev_ebitda', 
    'cashflow', 'profit_after_tax', 'deprec_cost', 'total_invest', 'fcff'
]

def fill_tb_value(code: str, df_invest: object):
    
    if len(df_invest.index) != 23:
        #print("Not enough VALUE field ", code)
        return
    
    for i in df_invest.columns:
        if 'IFRS' in i:
            continue
            
        ym = i
        values = []
        for j in range(len(df_invest[i])):
            #val = 'NULL' if type(df_invest[i][j]) != float or math.isnan(df_invest[i][j]) else df_invest[i][j]
            try:
                val = float(df_invest[i][j])
                if math.isnan(val):
                    val = 'NULL'
            except:
                val = 'NULL'
            
            values.append(val)
        
        eps, ebitdaps, cfps, sps, bps = values[1:6]
        dps, dps1, divpayout = values[7:10]
        per, pcr, psr, pbr, ev_sales, ev_ebitda = values[11:17]
        cashflow, profit_after_tax, deprec_cost, total_invest, fcff = values[18:]
        
        query = f'''
        INSERT INTO tb_value_indicator (
            code, year_month, eps, ebitdaps, cfps, sps, bps, 
            dps, dps1, divpayout, 
            per, pcr, psr, pbr, ev_sales, ev_ebitda, 
            cashflow, profit_after_tax, deprec_cost, total_invest, fcff
        )
        VALUES (
            "{code}", "{ym}", {eps}, {ebitdaps}, {cfps}, {sps}, {bps},
            {dps}, {dps1}, {divpayout}, 
            {per}, {pcr}, {psr}, {pbr}, {ev_sales}, {ev_ebitda}, 
            {cashflow}, {profit_after_tax}, {deprec_cost}, {total_invest}, {fcff}
        )
        '''
        cursor.execute(query)
        
    conn.commit()

In [16]:
import datetime
import requests
from bs4 import BeautifulSoup

totlen = len(all_stockinfo)

start_time = datetime.datetime.now()

for idx, info in enumerate(all_stockinfo):
#for idx, info in enumerate([['005930', '']]):
    code, _ = info
    
    #print(code)
    
    if idx % 100 == 0:
        print(f"===> {idx}")

    page = requests.get(fnguide_api.format(code))
    soup = BeautifulSoup(page.content, 'html')
    table_tags = soup.find_all('table')
    
    if len(table_tags) != 2:
        #print("Do this again: ", code, len(table_tags))
        quick_query = f'''select * from tb_stockinfo where code="{code}"'''
        cursor.execute(quick_query)
        #print(cursor.fetchall())
        
        continue
    
    df_price = pd.read_html(table_tags[0].prettify())[0]
    df_invest = pd.read_html(table_tags[1].prettify())[0]
    
    fill_tb_price(code, df_price)
    fill_tb_value(code, df_invest)
    
    
end_time = datetime.datetime.now()
print(end_time - start_time)


===> 0
===> 100
===> 200
===> 300
===> 400
===> 500
===> 600
===> 700
===> 800
===> 900
===> 1000
===> 1100
===> 1200
===> 1300
===> 1400
===> 1500
===> 1600
===> 1700
===> 1800
===> 1900
===> 2000
===> 2100
===> 2200
===> 2300
===> 2400
===> 2500
===> 2600
===> 2700
===> 2800
===> 2900
===> 3000
===> 3100
===> 3200
===> 3300
===> 3400
===> 3500
===> 3600
===> 3700
===> 3800
0:08:31.703023


# 조건에 부합하는 주식정보를 스크리닝 하여보자

In [17]:
cursor.execute('select * from tb_value_indicator where code = "005930" and year_month = "2019/12"')
cursor.fetchall()

[('005930',
  '2019/12',
  3166,
  8445,
  7523,
  33919,
  37528,
  1416,
  1417,
  44.73,
  17.63,
  7.42,
  1.65,
  1.49,
  1.64,
  6.6,
  494337,
  198361,
  295976,
  386970,
  107367)]

In [18]:
query = '''
SELECT
    info.comp_name AS `회사이름`
    ,info.code AS `종목코드`
    ,valu.eps AS `2019-12 EPS`
    ,valu.per AS `2019-12 PER`
    ,valu.pbr AS `2019-12 PBR`
    ,valu.profit_after_tax AS `2019-12 PROFIT`
    
FROM
    tb_stockinfo info
    
JOIN
    tb_value_indicator valu
ON
    info.code = valu.code
    
WHERE
    valu.year_month = "2019/12"
    
'''

df_spread = pd.read_sql_query(query, conn)

In [21]:
df_spread.head()

,회사이름,종목코드,2019-12 EPS,2019-12 PER,2019-12 PBR,2019-12 PROFIT
0,동북아12호선박투자,083370,300.0,16.67,1.00,5.0
1,에스지신성건설,001970,-2069.0,NaN,0.62,-97.0
2,웅진에너지,103130,-5033.0,NaN,NaN,-593.0
3,지에스엔텍,037640,95.0,7.35,0.08,94.0
4,한국체인,012350,1835.0,51.06,1.64,14.0


In [22]:
df_spread_es = df_spread.dropna()

In [24]:
len(df_spread.index), len(df_spread_es.index)

(2244, 1393)

## 이제 나만의 공식을 정립

[종목 선정시 확인하여야 하는 최소한의 지표](https://to-get-her.tistory.com/464)

> Bottom-up 방식으로 수익성 지표를 고를 떄
> - EPS, 증가율 20% 이상
> - ROE, 최근 몇년 내 15% 이상
> - 순이익률/영업이익률, 15% 이상


### ROE가 없네?
`ROE = EPS/BPS`
- ROE는 당기순익/자본총액
- EPS는 당기순익/총주식수
- BPS는 자본총액/총주식수
- [ROE 알아보기](https://md2biz.tistory.com/318)

### 순이익률이 없네?
`순이익률 = EPS/SPS`
- 개념상 순이익/매출액이므로,
- 주당 순이익인 EPS와
- 주당 매출액인 SPS를 활용해서,


In [38]:
query = '''
SELECT
    info.comp_name AS `회사이름`
    ,info.code AS `종목코드`
    ,val2019.eps AS `EPS 2019`
    ,val2019.bps AS `BPS 2019`
    ,val2019.sps AS `SPS 2019`
    ,val2018.eps AS `EPS 2018`
    ,val2018.bps AS `BPS 2018`
    ,val2017.eps AS `EPS 2017`
    ,val2017.bps AS `BPS 2017`
    ,val2016.eps AS `EPS 2016`
    ,val2016.bps AS `BPS 2016`
    
FROM
    tb_stockinfo info
    
JOIN
    tb_value_indicator val2019
ON
    info.code = val2019.code
AND
    val2019.year_month="2019/12"
    
JOIN
    tb_value_indicator val2018
ON
    info.code = val2018.code
AND
    val2018.year_month="2018/12"

JOIN
    tb_value_indicator val2017
ON
    info.code = val2017.code
AND
    val2017.year_month="2017/12"

JOIN
    tb_value_indicator val2016
ON
    info.code = val2016.code
AND
    val2016.year_month="2016/12"

'''

df_entire = pd.read_sql_query(query, conn)
df_entire = df_entire.dropna()
df_entire.head()

,회사이름,종목코드,EPS 2019,BPS 2019,SPS 2019,EPS 2018,BPS 2018,EPS 2017,BPS 2017,EPS 2016,BPS 2016
0,동북아12호선박투자,083370,300.0,5002.0,333.0,300.0,5002.0,300.0,5002.0,301.0,5002.0
1,에스지신성건설,001970,-2069.0,4826.0,18266.0,-1368.0,1286.0,932.0,2748.0,1017.0,1806.0
2,웅진에너지,103130,-5033.0,-3009.0,1349.0,-3637.0,1334.0,54.0,4281.0,-6150.0,3858.0
3,지에스엔텍,037640,95.0,8484.0,8604.0,163.0,8412.0,504.0,8431.0,100.0,7660.0
4,한국체인,012350,1835.0,56966.0,277467.0,655.0,57305.0,1201.0,56930.0,1231.0,56850.0


In [39]:
df_entire['ROE 2019'] = df_entire['EPS 2019'] / df_entire['BPS 2019']
df_entire['ROE 2018'] = df_entire['EPS 2018'] / df_entire['BPS 2018']
df_entire['ROE 2017'] = df_entire['EPS 2017'] / df_entire['BPS 2017']
df_entire['ROE 2016'] = df_entire['EPS 2016'] / df_entire['BPS 2016']
df_entire.head()

,회사이름,종목코드,EPS 2019,BPS 2019,SPS 2019,EPS 2018,BPS 2018,EPS 2017,BPS 2017,EPS 2016,BPS 2016,ROE 2019,ROE 2018,ROE 2017,ROE 2016
0,동북아12호선박투자,083370,300.0,5002.0,333.0,300.0,5002.0,300.0,5002.0,301.0,5002.0,0.059976,0.059976,0.059976,0.060176
1,에스지신성건설,001970,-2069.0,4826.0,18266.0,-1368.0,1286.0,932.0,2748.0,1017.0,1806.0,-0.428719,-1.063764,0.339156,0.563123
2,웅진에너지,103130,-5033.0,-3009.0,1349.0,-3637.0,1334.0,54.0,4281.0,-6150.0,3858.0,1.672649,-2.726387,0.012614,-1.594090
3,지에스엔텍,037640,95.0,8484.0,8604.0,163.0,8412.0,504.0,8431.0,100.0,7660.0,0.011198,0.019377,0.059779,0.013055
4,한국체인,012350,1835.0,56966.0,277467.0,655.0,57305.0,1201.0,56930.0,1231.0,56850.0,0.032212,0.011430,0.021096,0.021653


In [40]:
df_entire['EPS INC 2019'] = df_entire['EPS 2019']/df_entire['EPS 2018']
df_entire['EPS INC 2018'] = df_entire['EPS 2018']/df_entire['EPS 2017']
df_entire['EPS INC 2017'] = df_entire['EPS 2017']/df_entire['EPS 2016']
df_entire.head()

,회사이름,종목코드,EPS 2019,BPS 2019,SPS 2019,EPS 2018,BPS 2018,EPS 2017,BPS 2017,EPS 2016,BPS 2016,ROE 2019,ROE 2018,ROE 2017,ROE 2016,EPS INC 2019,EPS INC 2018,EPS INC 2017
0,동북아12호선박투자,083370,300.0,5002.0,333.0,300.0,5002.0,300.0,5002.0,301.0,5002.0,0.059976,0.059976,0.059976,0.060176,1.000000,1.000000,0.996678
1,에스지신성건설,001970,-2069.0,4826.0,18266.0,-1368.0,1286.0,932.0,2748.0,1017.0,1806.0,-0.428719,-1.063764,0.339156,0.563123,1.512427,-1.467811,0.916421
2,웅진에너지,103130,-5033.0,-3009.0,1349.0,-3637.0,1334.0,54.0,4281.0,-6150.0,3858.0,1.672649,-2.726387,0.012614,-1.594090,1.383833,-67.351852,-0.008780
3,지에스엔텍,037640,95.0,8484.0,8604.0,163.0,8412.0,504.0,8431.0,100.0,7660.0,0.011198,0.019377,0.059779,0.013055,0.582822,0.323413,5.040000
4,한국체인,012350,1835.0,56966.0,277467.0,655.0,57305.0,1201.0,56930.0,1231.0,56850.0,0.032212,0.011430,0.021096,0.021653,2.801527,0.545379,0.975630


In [41]:
df_entire['순이익률 2019'] = df_entire['EPS 2019'] / df_entire['SPS 2019']
df_entire.head()

,회사이름,종목코드,EPS 2019,BPS 2019,SPS 2019,EPS 2018,BPS 2018,EPS 2017,BPS 2017,EPS 2016,BPS 2016,ROE 2019,ROE 2018,ROE 2017,ROE 2016,EPS INC 2019,EPS INC 2018,EPS INC 2017,순이익률 2019
0,동북아12호선박투자,083370,300.0,5002.0,333.0,300.0,5002.0,300.0,5002.0,301.0,5002.0,0.059976,0.059976,0.059976,0.060176,1.000000,1.000000,0.996678,0.900901
1,에스지신성건설,001970,-2069.0,4826.0,18266.0,-1368.0,1286.0,932.0,2748.0,1017.0,1806.0,-0.428719,-1.063764,0.339156,0.563123,1.512427,-1.467811,0.916421,-0.113271
2,웅진에너지,103130,-5033.0,-3009.0,1349.0,-3637.0,1334.0,54.0,4281.0,-6150.0,3858.0,1.672649,-2.726387,0.012614,-1.594090,1.383833,-67.351852,-0.008780,-3.730912
3,지에스엔텍,037640,95.0,8484.0,8604.0,163.0,8412.0,504.0,8431.0,100.0,7660.0,0.011198,0.019377,0.059779,0.013055,0.582822,0.323413,5.040000,0.011041
4,한국체인,012350,1835.0,56966.0,277467.0,655.0,57305.0,1201.0,56930.0,1231.0,56850.0,0.032212,0.011430,0.021096,0.021653,2.801527,0.545379,0.975630,0.006613


## 스크리닝 결과

까먹어서 조건 다시!
- EPS, 증가율 20% 이상
- ROE, 최근 몇년 내 15% 이상
- 순이익률/영업이익률, 15% 이상


In [46]:
cond = (
    (df_entire['EPS INC 2019'] > 1.2) & (df_entire['EPS INC 2018'] > 1.2) & (df_entire['EPS INC 2017'] > 1.2)
    & (df_entire['ROE 2019'] > 1.15) & (df_entire['ROE 2018'] > 1.15) & (df_entire['ROE 2017'] > 1.15) & (df_entire['ROE 2016'] > 1.15)
    & (df_entire['순이익률 2019'] > 1.15)
)
sum(cond)

0

In [76]:
cond_rev = (
    (df_entire['EPS INC 2019'] > 0.5) & (df_entire['EPS INC 2018'] > 0.5) & (df_entire['EPS INC 2017'] > 0.5)
    & (df_entire['EPS 2019'] > 0) & (df_entire['EPS 2018'] > 0) & (df_entire['EPS 2017'] > 0) & (df_entire['EPS 2016'] > 0)
    & (df_entire['ROE 2019'] > 0.3) & (df_entire['ROE 2018'] > 0) & (df_entire['ROE 2017'] > 0) & (df_entire['ROE 2016'] > 0)
)
sum(cond_rev)

5

In [77]:
df_entire[cond_rev]

,회사이름,종목코드,EPS 2019,BPS 2019,SPS 2019,EPS 2018,BPS 2018,EPS 2017,BPS 2017,EPS 2016,BPS 2016,ROE 2019,ROE 2018,ROE 2017,ROE 2016,EPS INC 2019,EPS INC 2018,EPS INC 2017,순이익률 2019
236,푸드웰,005670,3109.0,7809.0,14394.0,450.0,4797.0,260.0,4462.0,154.0,4342.0,0.398130,0.093809,0.058270,0.035468,6.908889,1.730769,1.688312,0.215993
402,메디앙스,014100,3373.0,7228.0,8328.0,270.0,4533.0,507.0,4294.0,228.0,3813.0,0.466657,0.059563,0.118072,0.059795,12.492593,0.532544,2.223684,0.405019
416,삼양옵틱스,225190,1347.0,3929.0,6318.0,1246.0,3609.0,1784.0,3573.0,1597.0,3395.0,0.342835,0.345248,0.499300,0.470398,1.081059,0.698430,1.117095,0.213200
676,KG ETS,151860,2716.0,6840.0,4469.0,313.0,4277.0,326.0,4106.0,137.0,3864.0,0.397076,0.073182,0.079396,0.035455,8.677316,0.960123,2.379562,0.607742
1036,엔피디,198080,1167.0,3492.0,22370.0,621.0,2228.0,653.0,1755.0,1271.0,2312.0,0.334192,0.278725,0.372080,0.549740,1.879227,0.950995,0.513769,0.052168


# 선택된 주식 종목을 바탕으로 수익률을 계산해보자